In [54]:
from sqlalchemy import create_engine, text
import pandas as pd
import seaborn as sns
import mysql.connector
import missingno as msno
import pymysql

In [37]:
# df_1 = pd.read_csv("movie_metadata.csv")

# engine = create_engine('mysql+pymysql://root:@localhost:3306/imdb')
# conn = engine.connect()
# df_1.to_sql('movie', engine, if_exist='replace')

In [55]:
db_connection_info = {
    'user': 'root',
    'password': 'root',
    'host': '4.tcp.eu.ngrok.io',
    'port': 16148,
    'database': 'imdb'
}

db_url = f"mysql+mysqlconnector://{db_connection_info['user']}:{db_connection_info['password']}@{db_connection_info['host']}:{db_connection_info['port']}/{db_connection_info['database']}"

engine = create_engine(db_url)

connection = engine.connect()

query = text("SELECT * FROM movie")
result = connection.execute(query)

df = pd.read_sql(query, con=engine)

In [56]:
df

,index,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,4,None,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,...,NaN,None,None,None,NaN,NaN,12.0,7.1,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5038,5038,Color,Scott Smith,1.0,87.0,2.0,318.0,Daphne Zuniga,637.0,NaN,...,6.0,English,Canada,None,NaN,2013.0,470.0,7.7,NaN,84
5039,5039,Color,None,43.0,43.0,NaN,319.0,Valorie Curry,841.0,NaN,...,359.0,English,USA,TV-14,NaN,NaN,593.0,7.5,16.00,32000
5040,5040,Color,Benjamin Roberds,13.0,76.0,0.0,0.0,Maxwell Moody,0.0,NaN,...,3.0,English,USA,None,1400.0,2013.0,0.0,6.3,NaN,16
5041,5041,Color,Daniel Hsia,14.0,100.0,0.0,489.0,Daniel Henney,946.0,10443.0,...,9.0,English,USA,PG-13,NaN,2012.0,719.0,6.3,2.35,660
